<a href="https://colab.research.google.com/github/alvarofpinheiro/pifwia_aco/blob/main/PIFWIA_ACO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ant Colony Optimization (ACO) Encontrar a melhor rota (menor distância total) que percorre todos os pontos de um grafo gerado aleatoriamente e completamente conectado (cada ponto tem um caminho para todos os outros).

Kazharov, A.; Kureichik, V. Ant colony optimization algorithms for solving transportation problems. Journal of Computer and Systems Sciences International, Vol. 49. No. 1. pp. 30–43. https://doi.org/10.1134/S1064230710010053, 2010.


[Trilha de feromônio](https://upload.wikimedia.org/wikipedia/commons/thumb/a/af/Aco_branches.svg/2000px-Aco_branches.svg.png)

In [ ]:
#Bibliotecas
from numpy.random import choice
from scipy import spatial
import matplotlib.pyplot as plt
import random
import math

In [ ]:
# formiga é a unidades base da otimização, responsável por depositar o feromônio pelos caminhos que percorrem para indicar a qualidade do mesmo
class Formiga:
  def __init__(self, ponto_atual):
    self.ponto_atual = ponto_atual
    self.rota = [ponto_atual]
  def andar(self, ponto):
    self.ponto_atual = ponto
    self.rota.append(ponto)

# ponto é a representação de uma coordenada no espaço
class Ponto:
  def __init__(self, x, y):
    self.x = x
    self.y = y

# caminho é a ligação entre dois pontos
class Caminho:
  def __init__(self, ponto_i, ponto_j):
    self.ponto_i = ponto_i
    self.ponto_j = ponto_j
    self.comprimento = math.sqrt((ponto_i.x - ponto_j.x)**2 + (ponto_i.y - ponto_j.y)**2)
    self.feromonio = 0
    self.formigas_passantes = []
  def contem(self, formiga):
    if self.ponto_i == formiga.ponto_atual:
      return self.ponto_j not in formiga.rota
    elif self.ponto_j == formiga.ponto_atual:
      return self.ponto_i not in formiga.rota
    else:
      return False
  def ponto_adjacente(self, ponto):
    if self.ponto_i == ponto:
      return self.ponto_j
    elif self.ponto_j == ponto:
      return self.ponto_i
    else:
      return None

# grafo é a representação de um conjunto de pontos ligados por caminhos.
class Grafo:
  def __init__(self, caminhos):
    self.caminhos = caminhos
    self.melhor_rota = []
    self.comprimento_melhor_rota = 0
  def atualizas_melhor_rota(self, melhor_rota):
    self.melhor_rota = melhor_rota
    self.comprimento_melhor_rota = sum([math.sqrt((i.x - j.x)**2 + (i.y - j.y)**2) for [i, j] in melhor_rota])
  def possiveis_caminhos(self, formiga):
    return [caminho for caminho in self.caminhos if caminho.contem(formiga)]

In [ ]:
#Hiperparâmetros
f = 5 # quantidade de formigas na colônia
e = 0.2 # taxa de evaporação do feromônio nos caminhos
alfa = 0.5 # coeficente de importância da qtde feromônio para determinar probabilidade de escolha do caminho
beta = 0.7 # coeficente de importância do caminho para determinar a probabilidade de escolha do caminho
iteracoes = 15 # quantidade de iterações que irão acontecer até a parada da otimização
qtd_pontos = 5 # quantidade de pontos no grafo

In [ ]:
# criando os pontos
pontos = []
for _ in range(qtd_pontos):
  pontos.append(Ponto(random.uniform(-100, 100), random.uniform(-100, 100)))

# criando os caminhos
caminhos = []
i = 0
while i < qtd_pontos - 1:
  j = i + 1
  while j < qtd_pontos:
    caminhos.append(Caminho(pontos[i], pontos[j]))
    j += 1
  i += 1

# criando o grafo
grafo = Grafo(caminhos)
for ponto in pontos:
    plt.plot(ponto.x, ponto.y, marker='o', color='r')
x = []
y = []
for caminho in caminhos:
  x_i = caminho.ponto_i.x
  x_j = caminho.ponto_j.x
  y_i = caminho.ponto_i.y
  y_j = caminho.ponto_j.y
  x_texto = (x_i + x_j) / 2
  y_texto = (y_i + y_j) / 2
  plt.text(x_texto, y_texto, "{:.2f}".format(caminho.comprimento))
  x.append(x_i)
  x.append(x_j)
  y.append(y_i)
  y.append(y_j)

# plotando o gráfico
plt.plot(x, y, color='c')
plt.show()
gx = x
gy = y

In [ ]:
# número de formigas
def inicializar_colonia():
  formigas = []
  for _ in range(f):
    formigas.append(Formiga(random.choice(pontos)))
  return formigas

# escolhendo o caminho
def escolher_caminho(possiveis_caminhos):
  denominador = sum([(caminho.feromonio)**alfa * (1 / caminho.comprimento)**beta for caminho in possiveis_caminhos])
  distribuicao_probabilidades = None
  if denominador == 0:
    distribuicao_probabilidades = [1 / len(possiveis_caminhos)  for _ in possiveis_caminhos]
  else:
    distribuicao_probabilidades = [((caminho.feromonio)**alfa * (1 / caminho.comprimento)**beta) / denominador for caminho in possiveis_caminhos]
  return choice(possiveis_caminhos, 1, p=distribuicao_probabilidades)[0]

# calculando a rota
def distancia_rota(rota):
  distancia_rota = 0
  for i in range(0, len(rota) - 1):
    distancia = math.sqrt((rota[i].x - rota[i + 1].x)**2 + (rota[i].y - rota[i + 1].y)**2)
    distancia_rota += distancia
  return distancia_rota

# atualizando o feromônio
def atualizar_feromonios(caminhos):
  for caminho in caminhos:
    soma_heuristica = sum([1 / distancia_rota(formiga.rota) for formiga in caminho.formigas_passantes])
    caminho.feromonio = (1 - e) * caminho.feromonio + soma_heuristica # p = evaporação do feromônico
    caminho.formigas_passantes = []

# movimentando as formigas
def movimentar_formiga(formiga, grafo):
  while True:
    possiveis_caminhos = grafo.possiveis_caminhos(formiga)
    if possiveis_caminhos == []:
      break
    caminho_escolhido = escolher_caminho(possiveis_caminhos)
    caminho_escolhido.formigas_passantes.append(formiga)
    formiga.andar(caminho_escolhido.ponto_adjacente(formiga.ponto_atual))

In [ ]:
#Realiza a otimização exibindo os grafos
melhor_rota = None
distancia_melhor_rota = 0

# loop
for _ in range(iteracoes):
  print("Iteração: {:.0f}".format(_))
  formigas = inicializar_colonia()
  for formiga in formigas:
    movimentar_formiga(formiga, grafo)
    if melhor_rota is None or distancia_rota(melhor_rota) > distancia_rota(formiga.rota):
      melhor_rota = formiga.rota
      distancia_melhor_rota = distancia_rota(formiga.rota)
  atualizar_feromonios(grafo.caminhos)

  # mostrando a melhor rota a cada iteracao
  for ponto in pontos:
    plt.plot(ponto.x, ponto.y, marker='o', color='r')
  x = []
  y = []
  for caminho in caminhos:
    x_i = caminho.ponto_i.x
    x_j = caminho.ponto_j.x
    y_i = caminho.ponto_i.y
    y_j = caminho.ponto_j.y
    x_texto = (x_i + x_j) / 2
    y_texto = (y_i + y_j) / 2
    plt.text(x_texto, y_texto, "{:.2f}".format(caminho.comprimento))
    x.append(x_i)
    x.append(x_j)
    y.append(y_i)
    y.append(y_j)
  plt.plot(x, y, color='c')
  x = []
  y = []
  for ponto in melhor_rota:
    x.append(ponto.x)
    y.append(ponto.y)

  plt.plot(x, y, color='y')
  plt.show()
  print("Melhor rota: {:.2f}".format(distancia_melhor_rota))
